In [1]:
from data_science_toolkit.scraping.scraping import *
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
# search each link 
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import math
import re
import time
# lets open booking:

dfolder='./downloads'
geko_path='C:/Users/jaoss/Downloads/geckodriver/geckodriver.exe'
link='https://www.booking.com/index.en.html'


# browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)



In [28]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    
    #profile.add_extension('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/booking/ublock_origin-1.55.0.xpi')


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path, firefox_binary_path, download=True):
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder, download)
    options.binary_location = firefox_binary_path  # Set Firefox binary location
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
   
    # Enter the website address here
    browser.get(link)
   
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

dfolder='C:/Users/jaoss/Desktop/Data_Science/2nd_Term/Text_Mining/Assignment_1/booking_scraping/Downloads'
geko_path='C:/Users/jaoss/Downloads/geckodriver/geckodriver.exe'
link='https://www.booking.com'
firefox_binary_path = r'C:/Program Files/Mozilla Firefox/firefox.exe'  # Example path, adjust as needed


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path, firefox_binary_path=firefox_binary_path)

In [3]:
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath','//div[@class="d8f77e681c"]')
    #number of total hotel listings divided of listings p. page
    total_listings = int(re.findall(r'\d+', a[-1].text)[0])
    page_num = math.floor(total_listings / 25)
    print(f'Total hotels: {total_listings} \nNumber of pages: {page_num}')
    return(page_num) 

In [27]:
def search_hotels(start_date, end_date, place, browser):
    browser.get('https://www.booking.com')
    # Clear browsing history and cookies
    browser.delete_all_cookies()
    xpath = '//*[@id="onetrust-accept-btn-handler"]'

    time.sleep(10)
    try:
        check_and_click(browser, xpath, type='xpath')
    except:
        pass
    time.sleep(5)

    #accept cookies, refresh page then exit pop up (if it appears)
    browser.refresh()
    browser.refresh()
    browser.refresh()
    time.sleep(10)
    try:
        check_and_click(browser, '//button[@aria-label="Dismiss sign-in info."]', type='xpath')
    except:
    # Handle the exception without displaying the error message
        pass
    
    try:
        # Switch to the iframe and exit the google long in pop up
        iframe_path  = (By.XPATH, '//iframe[@title="Cuadro de diálogo Iniciar sesión con Google"]')
        iframe = WebDriverWait(browser, 10).until(EC.frame_to_be_available_and_switch_to_it(iframe_path))

        element_locator = (By.XPATH, '//div[@aria-label="Cerrar"]')
        element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable(element_locator))
        element.click()

        browser.switch_to.default_content()
    except Exception as e:
        pass

        
    search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
    search1.send_keys(place)

    #click date selector

    css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'

    browser.find_element('css selector',css).click()


    #click start and end date

    start_date_found = False
    end_date_found = False
    while start_date_found == False or end_date_found == False:
        path='//span[@class="cf06f772fa"]'
        dates = browser.find_elements('xpath',path)
        for date in dates:
            if date.get_attribute("data-date") == start_date:
                date.click()
                start_date_found = True
            if date.get_attribute("data-date") == end_date:
                date.click()
                end_date_found = True
                break
        next_page_button = '//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 f671049264 deab83296e f4552b6561 dc72a8413c f073249358"]'
        check_and_click(browser, next_page_button, type='xpath')
        time.sleep(3)

    time.sleep(3)

    #click search button
    my_xpath='/html/body/div[3]/div[2]/div/form/div[1]/div[4]/button'

    check_obscures(browser,my_xpath , type='xpath')
    check_and_click(browser,my_xpath , type='xpath')

    hotel_box_path = '//div[@data-testid="filters-group-label-content" and contains(text(), "Hotels")]'
    # hotel_box_path = '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[1]/div[3]/div[4]/div[8]/label/span[3]/div/div'
    hotel_boxes = browser.find_elements('xpath', hotel_box_path)

    if hotel_boxes:
        hotel_boxes[0].click()
    else:
        print("Element not found")
    time.sleep(5)

    # # Change language to english
    # lang_button = 'div.aca0ade214:nth-child(2) > span:nth-child(2) > button:nth-child(1)'
    # eng_button = 'div.ebac6e22e9 > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > ul:nth-child(1) > li:nth-child(3) > button:nth-child(1)'
    # browser.find_element('css selector',lang_button).click()
    # time.sleep(2)
    # browser.find_element('css selector',eng_button).click()
    # time.sleep(5)  # wait for language change to take effect

    pages = get_number_pages(browser)
    print(pages)

    hotel_card_container = '//div[@data-testid="property-card"]'  # XPath expression to locate the hotel card container

    #check "Hotels" box

    

        


    data = []  # List to store the extracted data for each hotel


    for page in range(pages+1):
        hotels = browser.find_elements('xpath', hotel_card_container)  # Find all hotel elements on each iteration
        for hotel in hotels:  # Iterate over each hotel element
            data_dic = {}  # Dictionary to store the extracted data for the current hotel

            try:
                # Extract the photo URL of the hotel
                photo_path = './/img[@data-testid="image"]'  # XPath expression to locate the photo element
                photo_url = hotel.find_element('xpath', photo_path).get_attribute('src')  # Get the src attribute of the photo element
                data_dic['photo_url'] = photo_url  # Store the photo URL in the dictionary
            except NoSuchElementException:
                print('No photo URL')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the link of the hotel
                link_path = './/a[@class="a78ca197d0"]'  # XPath expression to locate the link element
                link = hotel.find_element('xpath', link_path).get_attribute('href')  # Get the href attribute of the link element
                data_dic['link'] = link  # Store the link in the dictionary
            except NoSuchElementException:
                print('No link')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the name of the hotel
                name_path = './/div[@data-testid="title"]'  # XPath expression to locate the name element
                name = hotel.find_element('xpath', name_path).text  # Get the text content of the name element
                data_dic['name'] = name  # Store the name in the dictionary
            except NoSuchElementException:
                print('No name')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the central distance of the hotel
                central_distance_path = './/span[@data-testid="distance"]'  # XPath expression to locate the central distance element
                central_distance = hotel.find_element('xpath', central_distance_path).text  # Get the text content of the central distance element
                data_dic['central_distance'] = central_distance  # Store the central distance in the dictionary
            except NoSuchElementException:
                print('No central distance')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the sustainability information of the hotel
                sustainability_path = './/span[@class="abf093bdfe d068504c75 f68ecd98ea"]'  # XPath expression to locate the sustainability element
                sustainability = hotel.find_element('xpath', sustainability_path).text  # Get the text content of the sustainability element
                data_dic['sustainability'] = sustainability  # Store the sustainability information in the dictionary
            except NoSuchElementException:
                data_dic['sustainability'] = None
                print('No sustainability information')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the rating of the hotel
                rating_path = './/div[@class="a3b8729ab1 d86cee9b25"]'  # XPath expression to locate the rating element
                rating = hotel.find_element('xpath', rating_path).text  # Get the text content of the rating element
                data_dic['rating'] = rating  # Store the rating in the dictionary
            except NoSuchElementException:
                print('No rating')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the comfort rating of the hotel
                comfort_rating_path = './/span[@class="a3332d346a"]'  # XPath expression to locate the comfort rating element
                comfort_rating = hotel.find_element('xpath', comfort_rating_path).text  # Get the text content of the comfort rating element
                data_dic['comfort_rating'] = comfort_rating  # Store the comfort rating in the dictionary
            except NoSuchElementException:
                print('No comfort rating')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the review count of the hotel
                review_count_path = './/div[@class="abf093bdfe f45d8e4c32 d935416c47"]'  # XPath expression to locate the review count element
                review_count = hotel.find_element('xpath', review_count_path).text  # Get the text content of the review count element
                data_dic['review_count'] = review_count  # Store the review count in the dictionary
            except NoSuchElementException:
                print('No review count')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the room description of the hotel
                room_description_path = './/h4[@class="abf093bdfe e8f7c070a7"]'  # XPath expression to locate the room description element
                room_description = hotel.find_element('xpath', room_description_path).text  # Get the text content of the room description element
                data_dic['room_description'] = room_description  # Store the room description in the dictionary
            except NoSuchElementException:
                print('No room description')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the room specifics of the hotel
                room_specifics_path = './/div[@class="abf093bdfe"]'  # XPath expression to locate the room specifics elements
                room_specifics = hotel.find_elements('xpath', room_specifics_path)  # Find all room specifics elements
                room_specifics_text = []
                for room_specific in room_specifics:
                    room_specifics_text.append(room_specific.text)  # Get the text content of each room specifics element
                data_dic['room_specifics'] = ' '.join(room_specifics_text)  # Store the room specifics as a single string in the dictionary
            except NoSuchElementException:
                print('No room specifics')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the price of the hotel
                price_path = './/span[@data-testid="price-and-discounted-price"]'  # XPath expression to locate the price element
                price = hotel.find_element('xpath', price_path).text  # Get the text content of the price element
                data_dic['price'] = price  # Store the price in the dictionary
            except NoSuchElementException:
                print('No price')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the original price of the hotel (if available)
                original_price_path = './/span[@class="c73ff05531 e84eb96b1f"]'  # XPath expression to locate the original price element
                original_price = hotel.find_element('xpath', original_price_path).text  # Get the text content of the original price element
                data_dic['original_price'] = original_price  # Store the original price in the dictionary
            except NoSuchElementException:
                data_dic['original_price'] = None  # If the original price element is not found, store None in the dictionary
                print('No original price')
            except Exception as e:
                print(e)
                print('Something went wrong')

            try:
                # Extract the neighborhood and city of the hotel
                neighborhood_city_path = './/span[@data-testid="address"]'  # XPath expression to locate the neighborhood and city element
                neighborhood_city = hotel.find_element('xpath', neighborhood_city_path).text  # Get the text content of the neighborhood and city element
                data_dic['neighborhood_city'] = neighborhood_city  # Store the neighborhood and city in the dictionary
            except NoSuchElementException:
                print('No neighborhood and city')
            except Exception as e:
                print(e)
                print('Something went wrong')

            data.append(data_dic)  # Add the dictionary to the data list
            data_dic = {}
        # The data list now contains the extracted data for all hotels
        next_page_path = '//button[@aria-label="Next page"]'
        next_page = browser.find_elements('xpath', next_page_path)
        print(next_page)
        #element = WebDriverWait(browser, 30).until(EC.element_to_be_clickable(next_page[0]))
        next_page[0].click()
        time.sleep(3)
    data = pd.DataFrame(data)
    data['start_date'] = start_date
    data['end_date'] = end_date
    return data
        

In [5]:
def get_hotel_details(data):
    hotels_data = []
    x = 0 #just to tell how far the process is
    for link in data.link:
        hotel_data = {}
        print(link)
        x += 1
        print(f'{x} out of {len(data)}')
        r = requests.get(link)
        soup = BeautifulSoup(r.content, 'html.parser')
        id="hotel_sidebar_static_map"

        map_element = soup.find('a', id='hotel_sidebar_static_map').get('data-atlas-latlng')
        hotel_data['map_lat'] = map_element.split(',')[0]
        hotel_data['map_lng'] = map_element.split(',')[1]

        hotel_data['name'] = soup.find('h2', class_='d2fee87262 pp-header__title').get_text()
        #hotel_data['address'] = soup.find('span', class_='hp_address_subtitle').get_text()
        hotel_data['description'] = soup.find('div', class_='hp-description').get_text()

        # room_services = soup.find_all('span', class_='a5a5a75131')
        # hotel_data['room_services'] = ', '.join([room_service.get_text() for room_service in room_services])
        
        # room_ratings = soup.find_all('div', class_='ccb65902b2')
        # room_ratings_desc = [element.get_text() for element in room_ratings if ',' not in element.get_text()]
        # room_ratings = [element.get_text() for element in room_ratings if ',' in element.get_text()]


        # for index in range(0,len(room_ratings)): 
        #     hotel_data[f'room_rating_{room_ratings_desc[index]}'] = room_ratings[index]

    
        #hotel_data['room_options_count'] = len(soup.find_all('tr', class_='js-rt-block-row'))

        hotels_data.append(hotel_data)
    hotels_data = pd.DataFrame(hotels_data)
    return pd.concat([data, hotels_data], axis=1)


In [6]:
def clean_df(df):
    df['central_distance'] = df['central_distance'].str.extract(r'(\d+,*\d*)')
    df['central_distance'] = df['central_distance'].str.replace(',', '.').astype(float)
    # TODO clean outliers

    df['price'] = df['price'].str.extract(r'(\d+.*\d*)').astype(float)

    df['rating'] = df['rating'].str.replace(',','.').astype(float)

    df['review_count'] = df['review_count'].str.extract(r'(\d+\.*\d*)')
    df['review_count'] = df['review_count'].str.replace('.','').astype(float)
    
    df = df.drop(columns=['photo_url', 'link', 'sustainability', 'original_price']) 
      # 'room_rating_Personal ', 'room_rating_Instalaciones y servicios ',
      #  'room_rating_Limpieza ', 'room_rating_Confort ',
      #  'room_rating_Relación calidad-precio ', 'room_rating_Ubicación ',
      #  'room_rating_WiFi gratis ', 'comfort_rating'])
    
    return df

In [29]:
start_date = '2024-04-14'
end_date = '2024-04-21'
place = "San Francisco, California, United States"
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data = get_hotel_details(data)
# data = clean_df(data)
data.to_csv(file_path)

Message: Unable to locate element: //button[@aria-label="Dismiss sign-in info."]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: //button[@aria-label="Dismiss sign-in info."]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find

In [21]:
data.head()

,photo_url,link,name,central_distance,sustainability,rating,comfort_rating,review_count,room_description,room_specifics,price,original_price,neighborhood_city,start_date,end_date,map_lat,map_lng,name,description
0,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/us/caza.html?lab...,Hotel Caza Fisherman's Wharf,2.3 km from center,Travel Sustainable Level 2,8.4,Comfort 9.0,"2,940 reviews",Deluxe King Room,1 king bed,"€ 1,221","€ 1,626","Fisherman's Wharf, San Francisco",2024-03-03,2024-03-10,37.80658700,-122.41801300,Hotel Caza Fisherman's Wharf,\n\n\n\n\nYou're eligible for a Genius discoun...
1,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/us/baldwinhotels...,Hotel 32One,300 m from center,Travel Sustainable Level 2,8.1,Comfort 8.6,"2,165 reviews",Standard Room - Accessible,1 full bed,"€ 1,236","€ 1,359","Union Square, San Francisco",2024-03-03,2024-03-10,37.78998273,-122.40561850,Hotel 32One,\n\n\n\n\nYou're eligible for a Genius discoun...
2,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/us/the-mosser-sa...,The Mosser Hotel,350 m from center,Travel Sustainable Level 2,7.2,NaN,"1,755 reviews",Double Room with Shared Bathroom,1 full bed,€ 439,€ 965,"Union Square, San Francisco",2024-03-03,2024-03-10,37.78501623,-122.40513034,The Mosser Hotel,"\n\n\n\n\nThis San Francisco hotel, built in 1..."
3,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/us/riu-plaza-fis...,Riu Plaza Fisherman's Wharf,2.2 km from center,Travel Sustainable Level 3+,8.2,Comfort 8.8,"11,333 reviews",Deluxe King Room,1 king bed,"€ 1,431","€ 2,347","Fisherman's Wharf, San Francisco",2024-03-03,2024-03-10,37.80699906,-122.41348274,Riu Plaza Fisherman's Wharf,\n\n\n\n\nYou're eligible for a Genius discoun...
4,https://cf.bstatic.com/xdata/images/hotel/squa...,https://www.booking.com/hotel/us/the-bartlett-...,The Bartlett Hotel and Guesthouse,200 m from center,Travel Sustainable Level 1,7.2,NaN,"1,666 reviews",Queen Room with Shared Bathroom,1 queen bed,€ 462,None,"Union Square, San Francisco",2024-03-03,2024-03-10,37.78639728,-122.40875090,The Bartlett Hotel and Guesthouse,\n\n\n\n\nYou're eligible for a Genius discoun...


In [125]:
start_date = '2024-04-14'
end_date = '2024-04-21'
place = "Los Ángeles, California, Estados Unidos"
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_c1_w2 = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_c1_w2 = get_hotel_details(data_c1_w2)

Message: Unable to locate element: //*[@id="onetrust-accept-btn-handler"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: //*[@id="onetrust-accept-btn-handler"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://

In [126]:
start_date = '2024-03-03'
end_date = '2024-03-10'
place = "San Francisco, California, United States"
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_c2_w1 = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_c2_w1 = get_hotel_details(data_c2_w1)

Message: Unable to locate element: //*[@id="onetrust-accept-btn-handler"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: //*[@id="onetrust-accept-btn-handler"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://

MissingSchema: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?

In [ ]:
start_date = '2024-04-14'
end_date = '2024-04-21'
place = "San Francisco, California, United States"
file_path = start_date +'_'+end_date+'_'+place+'.csv'
data_c2_w2 = search_hotels(browser=browser, start_date=start_date,end_date=end_date,place=place)
data_c2_w2 = get_hotel_details(data_c2_w2)